# Importe as bibliotecas necessárias

In [1]:
'''
# Instalando as bibliotecas necessárias
%pip install -U pip setuptools wheel
%pip install pymongo
%pip install pandas
%pip install nltk
%pip install wordcloud
%pip install spacy
%pip install matplotlib
%pip install numpy==1.26.4
%pip install -U scikit-learn
%pip install unidecode
'''

'\n# Instalando as bibliotecas necessárias\n%pip install -U pip setuptools wheel\n%pip install pymongo\n%pip install pandas\n%pip install nltk\n%pip install wordcloud\n%pip install spacy\n%pip install matplotlib\n%pip install numpy==1.26.4\n%pip install -U scikit-learn\n%pip install unidecode\n'

# Importando as bibliotecas

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
from datetime import datetime
from db import connection_db as conndb
from db import filters
from utils import plot_graphs
from utils.text_treatment import TextTreatment
from utils.text_vectorization import TextVectorization
from dateutil.relativedelta import relativedelta
from utils.nuvem_palavras import NuvemPalavras
from tqdm.auto import tqdm

tqdm.pandas()

In [3]:
mongo_connection = conndb.MongoDBConnection(uri='mongodb://localhost:27017/', database_name='dadosVivamente',
                                            collection_name='dadosSemFiltros')
mongo_connection.connect()
collection = mongo_connection.collection

Conexão estabelecida com sucesso ao banco de dados.


In [4]:
collection_filters = filters.CollectionFilters(collection)
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')
collection_filters.apply_pipeline2(7, 2, 'posts7anos2anos')
collection_filters.apply_pipeline3('postsComBDIAndInfos')
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')
data_inicio = datetime(2017, 12, 1)
data_fim = data_inicio - relativedelta(months=6)
collection_filters.apply_pipeline5('postsFiltradosPorData', data_inicio, data_fim)
collection_filters.quant_users_cat('suicida', '$eq', '3')
collection_filters.count_users_by_gender('suicida', '$eq', '3', 'M')
collection_filters.count_users_by_gender('suicida', '$eq', '3', 'F')
collection = collection_filters.collection

INFO:root:A coleção já existe: dadosComFiltrosIniciais
INFO:root:A coleção já existe: posts7anos2anos
INFO:root:A coleção já existe: postsComBDIAndInfos
INFO:root:A coleção já existe: postsComBDIAndInfosFiltroDataPosts
INFO:root:A coleção já existe: postsFiltradosPorData


Quantidade de Usuários: 58
Quantidade de usuários M: 11
Quantidade de usuários F: 47


In [5]:
# Filtrando documentos com o atributo maior que 3
filtro = {"nivel": {"$eq": 2}}
documentos = collection.find(filtro)

In [6]:
# Transformando os documentos em um DataFrame
df = pd.DataFrame(list(documentos))
df.head()

,_id,idade,sexo,id_usuario,nivel,pessimismo,tristeza,fracasso,prazer,culpa,...,quantAmigos,postMessage,postStory,postCreated_time,diaDaSemana,hora,minutos,diaDoMes,mes,ano
0,59c325925fef19001e03ae70,33,F,1358116567556711,2.0,1,1,0,0,1,...,1099,❤❤❤,Elisa Lacal shared a memory.,2017-09-20 13:43:35,Quarta-feira,13,43,20,9,2017
1,59c325925fef19001e03ae71,33,F,1358116567556711,2.0,1,1,0,0,1,...,1099,NaN,Elisa Lacal shared Colégio Master :: Oficial :...,2017-09-16 10:23:09,Sábado,10,23,16,9,2017
2,59c325925fef19001e03ae72,33,F,1358116567556711,2.0,1,1,0,0,1,...,1099,NaN,Ellen Bueno added 4 new photos — with Marcus A...,2017-09-10 22:09:50,Domingo,22,9,10,9,2017
3,59c325925fef19001e03ae73,33,F,1358116567556711,2.0,1,1,0,0,1,...,1099,#pescadores_br #caceres #riosepotuba #fishing,Murilo Alves Gonçalves Dos Santos is with Marc...,2017-09-10 20:05:32,Domingo,20,5,10,9,2017
4,59c325925fef19001e03ae74,33,F,1358116567556711,2.0,1,1,0,0,1,...,1099,Parabéns Giulia!! O tão esperado #DiaDaAlta ch...,REDEORTO Várzea Grande is feeling blissful wit...,2017-09-05 20:05:46,Terça-feira,20,5,5,9,2017


In [7]:
print(df.dtypes)
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

_id                         object
idade                        int64
sexo                        object
id_usuario                  object
nivel                      float64
pessimismo                  object
tristeza                    object
fracasso                    object
prazer                      object
culpa                       object
punicao                     object
estima                      object
critica                     object
suicida                     object
choro                       object
agitacao                    object
interesse                   object
indecisao                   object
desvalorizacao              object
energia                     object
sono                        object
irritabilidade              object
apetite                     object
concentracao                object
fadiga                      object
int_sexo                    object
quantAmigos                 object
postMessage                 object
postStory           

In [8]:
# Salvar o DataFrame em um arquivo CSV
df.to_csv('dados/com_filtros_datas/6meses/so_suicida_6_meses.csv', index=False)

In [9]:
# Agrupar por usuário, mês e ano
posts_grouped = df.groupby(['id_usuario', 'mes', 'ano']).size().reset_index(name='quantidade')

# Adicionar coluna com o período e converter para datetime
posts_grouped['periodo'] = pd.to_datetime(posts_grouped['mes'].astype(str) + '/' + posts_grouped['ano'].astype(str),
                                          format='%m/%Y')

# Deve retornar 0 se a conversão foi bem-sucedida.
print(posts_grouped['periodo'].isnull().sum())

posts_grouped.dtypes

0


id_usuario            object
mes                    int64
ano                    int64
quantidade             int64
periodo       datetime64[ns]
dtype: object

# Plotar quantidade de posts por usuário

In [10]:
# plot_graphs.PlotGraphs().plot_posts_per_user(posts_grouped, 'dados/com_filtros_datas/6meses/graficos')

# Efentuando a limpeza dos dados

In [11]:
tratamento_texto = TextTreatment()

/home/paulovss/Documentos/projects/python/analise_dados/venv/lib/python3.12/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental featur

In [12]:
df['postMessage'] = df['postMessage'].fillna('').astype(str)
resultado = tratamento_texto.finds_pandas_words(df['postMessage'], 'dados/datasets/nomes_medicamentos_antidepressivos.txt')
print(resultado)

0        False
1        False
2        False
3        False
4        False
         ...  
34509    False
34510    False
34511    False
34512    False
34513    False
Name: postMessage, Length: 34514, dtype: bool


In [13]:
tratamento_texto.finds_fuzzy_words(df['postMessage'], 'dados/datasets/nomes_medicamentos_antidepressivos.txt')

[]

In [ ]:
df['postMessageLimpo'] = df['postMessage'].fillna('').progress_apply(
    lambda texto: tratamento_texto.preprocessamento_texto(texto) if texto else '')

In [ ]:
# Exibir apenas as colunas postMessage e postMessageLimpo
df[['postMessage', 'postMessageLimpo']].head()

In [ ]:
# Carregar stopwords em português para os TF-IDF e Bag of Words
stop_words = stopwords.words('portuguese')

In [ ]:
text_vectorizer = TextVectorization(df, stop_words)
# Vetorização TF-IDF
resultados_tfidf = text_vectorizer.tfidf_vectorization(
    'dados/com_filtros_datas/6meses/so_suicida_resultados_tfidf_unigramas.csv', 0.60, 2,
    (1, 1))

# Vetorização Bag of Words
resultados_bow = text_vectorizer.bag_of_words_vectorization(
    'dados/com_filtros_datas/6meses/so_suicida_resultados_bow_unigramas.csv',
    0.85, 5, (1, 1))

In [ ]:
text_vectorizer.plot_top_words(resultados_tfidf, column='score', title='Top 10 Palavras TF-IDF')
text_vectorizer.plot_top_words(resultados_bow, column='contagem', title='Top 10 Palavras Bag of Words')

In [ ]:
frequencias_bow = dict(zip(resultados_bow['palavra'], resultados_bow['contagem']))

# Gerar e plotar a nuvem de palavras
NuvemPalavras.plot_nuvem_palavras(frequencias_bow)

In [ ]:
frequencias_tfidf = dict(zip(resultados_tfidf['palavra'], resultados_tfidf['score']))

# Gerar e plotar a nuvem de palavras
NuvemPalavras.plot_nuvem_palavras(frequencias_tfidf)